### To run this file local spark is required. 
please feel free to use : https://github.com/BabakEA/Install_Apache_SPARK_Hadoop_on_Windows_Machine  to install local apache spark.


In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf,SQLContext,HiveContext
from pyspark.sql import SparkSession,DataFrame,Column,Row,GroupedData,DataFrameNaFunctions
from pyspark.sql.functions import lit,concat,col,concat_ws
from pyspark.sql import functions as sf

#from local_lib.PSPARK_DF_245 import selectExpr

import argparse


# load Dataset for proccess     

from IPython.display import clear_output

import sched, time
import os
import pandas as pd
import numpy as np
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
pd.options.display.float_format = '{:.6f}'.format


import uszipcode


C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:

try: 
    sc.stop()
except:
    pass


conf = SparkConf().setAppName("PY_DF").setMaster("local[*]").set("spark.executor.memory", "10g")
sc = SparkContext(conf = conf)

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("AI_FOREST") \
    .config("spark.executor.memory", "10g") \
    .getOrCreate()

hc = HiveContext(sc)
sqlContext = SQLContext(sc)

In [5]:
df1 = spark.read.csv('./data/green_tripdata_2016-02.csv', header=True)
df= df1.select("lpep_pickup_datetime","Lpep_dropoff_datetime",
         "Pickup_longitude","Pickup_latitude",
         "Dropoff_longitude","Dropoff_latitude",
         "Trip_distance")
df



DataFrame[lpep_pickup_datetime: string, Lpep_dropoff_datetime: string, Pickup_longitude: string, Pickup_latitude: string, Dropoff_longitude: string, Dropoff_latitude: string, Trip_distance: string]

In [12]:
len(df1.columns)
df1.count()

1510722

In [15]:
print("""Data shape : Number of  columns : {COL} 
      and Number of rows : {ROW}""".format(COL= len(df1.columns), ROW =df1.count()))     

Data shape : Number of  columns : 21 
      and Number of rows : 1510722


In [4]:
pdf= df[["lpep_pickup_datetime","Lpep_dropoff_datetime",
         "Pickup_longitude","Pickup_latitude",
         "Dropoff_longitude","Dropoff_latitude",
         "Trip_distance"]].toPandas()

pdf["lpep_pickup_datetime"]=pd.to_datetime(pdf["lpep_pickup_datetime"])
pdf["Lpep_dropoff_datetime"]=pd.to_datetime(pdf["Lpep_dropoff_datetime"])

for col in ["Pickup_longitude","Pickup_latitude","Dropoff_longitude","Dropoff_latitude","Trip_distance"]:
    pdf[col]=pd.to_numeric(pdf[col], downcast="float").round(6)
    pdf[col]=pdf[col].astype(float).round(6)
    








In [5]:
pdf

,lpep_pickup_datetime,Lpep_dropoff_datetime,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Trip_distance
0,2016-02-01 00:00:01,2016-02-01 00:10:06,-73.939018,40.805210,-73.972534,40.785885,2.860000
1,2016-02-01 00:01:33,2016-02-01 00:20:13,-73.891495,40.746651,-73.890877,40.743896,3.350000
2,2016-02-01 00:03:46,2016-02-01 00:21:04,-73.983772,40.676132,-73.956978,40.718327,4.700000
3,2016-02-01 00:00:05,2016-02-01 00:06:48,-73.807518,40.700375,-73.831657,40.705978,2.110000
4,2016-02-01 00:06:20,2016-02-01 00:08:47,-73.903961,40.744938,-73.900009,40.733601,0.980000
...,...,...,...,...,...,...,...
1510717,2016-02-29 23:00:49,2016-02-29 23:17:52,-73.867195,40.752384,-73.877914,40.756538,2.100000
1510718,2016-02-29 23:00:46,2016-02-29 23:10:08,-73.916656,40.757957,-73.881950,40.755867,1.900000
1510719,2016-02-29 23:00:17,2016-02-29 23:15:02,-73.979424,40.688305,-73.923569,40.698891,3.300000
1510720,2016-02-29 23:00:02,2016-02-29 23:06:22,-73.987488,40.702381,-73.995041,40.719868,2.200000


In [6]:
import pandas as pd
import numpy as np
import time
from uszipcode import SearchEngine

search = SearchEngine()

In [7]:
pdf["PICK_ZIP"]=np.nan

In [8]:
tic = time.time()
for idx in pdf.iloc[0:10,:].index:
    try:
        df['PICK_ZIP'][idx] = search.by_coordinates(
                   lat=float(pdf['latitude'][idx]),lng= float(pdf['longitude'][idx]), 
                    returns=1)[0].to_dict()["zipcode"]
    except:
        pdf['PICK_ZIP'][idx] = np.nan
toc = time.time()

print(toc-tic)

0.07028412818908691


In [9]:
# Regarding the API limitation , Pandas multi processing didn't worked 

pdf.head(10)

,lpep_pickup_datetime,Lpep_dropoff_datetime,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Trip_distance,PICK_ZIP
0,2016-02-01 00:00:01,2016-02-01 00:10:06,-73.939018,40.805210,-73.972534,40.785885,2.860000,nan
1,2016-02-01 00:01:33,2016-02-01 00:20:13,-73.891495,40.746651,-73.890877,40.743896,3.350000,nan
2,2016-02-01 00:03:46,2016-02-01 00:21:04,-73.983772,40.676132,-73.956978,40.718327,4.700000,nan
3,2016-02-01 00:00:05,2016-02-01 00:06:48,-73.807518,40.700375,-73.831657,40.705978,2.110000,nan
4,2016-02-01 00:06:20,2016-02-01 00:08:47,-73.903961,40.744938,-73.900009,40.733601,0.980000,nan
5,2016-02-01 00:00:42,2016-02-01 00:27:25,-73.978615,40.670422,-73.986893,40.748573,6.000000,nan
6,2016-02-01 00:00:59,2016-02-01 00:07:25,-73.829826,40.759659,-73.809982,40.751160,1.460000,nan
7,2016-02-01 00:00:56,2016-02-01 00:15:31,-73.955017,40.735298,-73.939865,40.793724,5.960000,nan
8,2016-02-01 00:02:45,2016-02-01 00:09:19,-73.959496,40.718136,-73.948105,40.709122,1.440000,nan
9,2016-02-01 00:04:12,2016-02-01 00:19:36,-73.944565,40.714516,-73.918343,40.689548,2.580000,nan


# Get a slice of Pyspark Dataframe to test the data

In [21]:
df1=df.limit(100)
df1.show()


+--------------------+---------------------+-------------------+------------------+-------------------+------------------+-------------+
|lpep_pickup_datetime|Lpep_dropoff_datetime|   Pickup_longitude|   Pickup_latitude|  Dropoff_longitude|  Dropoff_latitude|Trip_distance|
+--------------------+---------------------+-------------------+------------------+-------------------+------------------+-------------+
| 2016-02-01 00:00:01|  2016-02-01 00:10:06|-73.939018249511719|40.805213928222656|  -73.9725341796875|40.785884857177734|         2.86|
| 2016-02-01 00:01:33|  2016-02-01 00:20:13|-73.891494750976563|40.746650695800781|-73.890876770019531|   40.743896484375|         3.35|
| 2016-02-01 00:03:46|  2016-02-01 00:21:04|-73.983779907226562|40.676132202148438|-73.956977844238281|40.718326568603516|         4.70|
| 2016-02-01 00:00:05|  2016-02-01 00:06:48|-73.807518005371094|40.700374603271484|-73.831657409667969|40.705978393554687|         2.11|
| 2016-02-01 00:06:20|  2016-02-01 00:08:

### Using Pyspark udf to apply API to convert geo points to zipcode


In [25]:
from pyspark.sql.functions import udf, col

@udf('string')
def get_zip_udf2(latitude, longitude):
    search = SearchEngine()
    try:
        zip = search.by_coordinates(float(latitude), float(longitude), returns=1)[0].to_dict()["zipcode"]
    except:
        zip = np.nan
    return zip

  
#df1.withColumn('zip', get_zip_udf2(col("Pickup_latitude"),col("Pickup_longitude"))).show()
df=df.withColumn('PICKUP_ZIP', get_zip_udf2(col("Pickup_latitude"),col("Pickup_longitude")))
df=df.withColumn('DROP_ZIP', get_zip_udf2(col("Dropoff_latitude"),col("Dropoff_longitude")))



In [27]:
%time pdf=df.toPandas()

Wall time: 4h 14min 14s


In [30]:
pdf.to_csv("./data/updated_RAW_TAXI.csv")

In [29]:
pdf

,lpep_pickup_datetime,Lpep_dropoff_datetime,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Trip_distance,PICKUP_ZIP,DROP_ZIP
0,2016-02-01 00:00:01,2016-02-01 00:10:06,-73.939018249511719,40.805213928222656,-73.9725341796875,40.785884857177734,2.86,10035,10162
1,2016-02-01 00:01:33,2016-02-01 00:20:13,-73.891494750976563,40.746650695800781,-73.890876770019531,40.743896484375,3.35,11372,11373
2,2016-02-01 00:03:46,2016-02-01 00:21:04,-73.983779907226562,40.676132202148438,-73.956977844238281,40.718326568603516,4.70,11217,11211
3,2016-02-01 00:00:05,2016-02-01 00:06:48,-73.807518005371094,40.700374603271484,-73.831657409667969,40.705978393554687,2.11,11435,11351
4,2016-02-01 00:06:20,2016-02-01 00:08:47,-73.903961181640625,40.74493408203125,-73.900009155273438,40.733600616455078,.98,11377,11378
...,...,...,...,...,...,...,...,...,...
1510717,2016-02-29 23:00:49,2016-02-29 23:17:52,-73.867195129394531,40.752384185791016,-73.877914428710937,40.756538391113281,2.10,11369,11372
1510718,2016-02-29 23:00:46,2016-02-29 23:10:08,-73.916656494140625,40.757957458496094,-73.881950378417969,40.755867004394531,1.90,11103,11372
1510719,2016-02-29 23:00:17,2016-02-29 23:15:02,-73.979423522949219,40.688304901123047,-73.923568725585938,40.698890686035156,3.30,11217,11237
1510720,2016-02-29 23:00:02,2016-02-29 23:06:22,-73.98748779296875,40.702381134033203,-73.995040893554687,40.719867706298828,2.20,11201,10002


In [31]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1510722 entries, 0 to 1510721
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   lpep_pickup_datetime   1510722 non-null  object
 1   Lpep_dropoff_datetime  1510722 non-null  object
 2   Pickup_longitude       1510722 non-null  object
 3   Pickup_latitude        1510722 non-null  object
 4   Dropoff_longitude      1510722 non-null  object
 5   Dropoff_latitude       1510722 non-null  object
 6   Trip_distance          1510722 non-null  object
 7   PICKUP_ZIP             1510722 non-null  object
 8   DROP_ZIP               1510722 non-null  object
dtypes: object(9)
memory usage: 103.7+ MB


In [32]:
pdf["lpep_pickup_datetime"]=pd.to_datetime(pdf["lpep_pickup_datetime"])
pdf["Lpep_dropoff_datetime"]=pd.to_datetime(pdf["Lpep_dropoff_datetime"])

for col in ["Pickup_longitude",'Pickup_latitude','Dropoff_longitude','Dropoff_latitude','Trip_distance','PICKUP_ZIP','DROP_ZIP']:
    pdf[col]=pdf[col].astype(float)





In [33]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1510722 entries, 0 to 1510721
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   lpep_pickup_datetime   1510722 non-null  datetime64[ns]
 1   Lpep_dropoff_datetime  1510722 non-null  datetime64[ns]
 2   Pickup_longitude       1510722 non-null  float64       
 3   Pickup_latitude        1510722 non-null  float64       
 4   Dropoff_longitude      1510722 non-null  float64       
 5   Dropoff_latitude       1510722 non-null  float64       
 6   Trip_distance          1510722 non-null  float64       
 7   PICKUP_ZIP             1508154 non-null  float64       
 8   DROP_ZIP               1508365 non-null  float64       
dtypes: datetime64[ns](2), float64(7)
memory usage: 103.7 MB
